In [1]:
import numpy as np # linear algebra
import pandas as pd # pandas for dataframe based data processing and CSV file I/O

In [2]:
import requests # for http requests
from bs4 import BeautifulSoup # for html parsing and scraping
from fastnumbers import isfloat 
from fastnumbers import fast_float
from multiprocessing.dummy import Pool as ThreadPool 
import bs4

import matplotlib.pyplot as plt
import seaborn as sns
import json
from tidylib import tidy_document # for tidying incorrect html

sns.set_style('whitegrid')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:


def ffloat(string):
    if string is None:
        return np.nan
    if type(string)==float or type(string)==np.float64:
        return string
    if type(string)==int or type(string)==np.int64:
        return string
    return fast_float(string.split(" ")[0].replace(',','').replace('%',''),
                      default=np.nan)

def ffloat_list(string_list):
    return list(map(ffloat,string_list))

def remove_multiple_spaces(string):
    if type(string)==str:
        return ' '.join(string.split())
    return string


In [4]:
def request_with_check(url):
    page_response = requests.get(url, timeout=240)
    status = page_response.status_code
    if status>299:
        raise AssertionError("page content not found, status: %s"%status)
    return page_response

In [5]:
from IPython.core.display import HTML

page_response = requests.get("https://www.screener.in/company/INFY/", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))
#print(page_content)

HTML(str(page_content.find("p",attrs={'class':"sub bigger"})))
#print(str(page_content.find("p",attrs={'class':"sub bigger"})))
HTML(str(page_content.find("li",attrs={'class':"four columns"})))
#print(str(page_content.find("li",attrs={'class':"four columns"})))
market_cap_info = page_content.find("li",attrs={'class':"four columns"})
#print(str(market_cap_info.find("b")))

#targetFile = open('page_content.txt', 'w')
#targetFile.write(str(page_content))

ratios_section = page_content.find("section",attrs={'id':"ratios"})
print(ratios_section.h2)

In [6]:
page_response = requests.get("https://www.screener.in/company/INFY/", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))
#print(page_content)

#HTML(str(page_content.find("div",attrs={'class':"company-profile-about"}).find("p")))
#print(str(page_content.find("p",attrs={'class':"sub bigger"})))
#HTML(str(page_content.find("li",attrs={'class':"four columns"})))
#print(str(page_content.find("li",attrs={'class':"four columns"})))
market_cap_info = page_content.find("div",attrs={'class':"company-ratios"}).find("span",attrs={'class':"nowrap value"})#.find("span",attrs={'class':"number"})
print(market_cap_info.find("span").string)

#print(str(market_cap_info))
#print(str(market_cap_info.find("span")))

#targetFile = open('page_content.txt', 'w')
#targetFile.write(str(page_content))

#ratios_section = page_content.find("section",attrs={'id':"ratios"})
#print(ratios_section.h2)


tag_str = str(page_content.find("li",attrs={'class':"flex flex-space-between"}))
if "Market Cap" in tag_str:
    marcap = BeautifulSoup(tag_str)
    print(str(marcap.find("span",attrs={'class':"number"}).string) + " Cr.")
else:
    print("hello")

472,423
472,423 Cr.


In [7]:
#print(page_content)
def findFullDescription_Screener(page_content):
    tag = page_content.find("div",attrs={'class':"company-profile-about"}).find("p")
    return tag.string

def findMarketCap_Screener(page_content):
    tag_str = str(page_content.find("li",attrs={'class':"flex flex-space-between"}))
    if "Market Cap" in tag_str:
        marcap = BeautifulSoup(tag_str)
        return (str(marcap.find("span",attrs={'class':"number"}).string) + " Cr.")
    else:
        return "O Cr."


In [8]:
def get_table_simple(table,is_table_tag=True):
    elems = table.find_all('tr') if is_table_tag else get_children(table)
    table_data = list()
    for row in elems:
        
        row_data = list()
        row_elems = get_children(row)
        for elem in row_elems:
            text = elem.text.strip().replace("\n","").replace(",","")
            text = remove_multiple_spaces(text)
            if len(text)==0:
                continue
            row_data.append(text)
        table_data.append(row_data)
    return table_data
    
def get_children(html_content):
    children = list()
    for item in html_content.children:
        if type(item)==bs4.element.Comment:
            continue
        if type(item)==bs4.element.Tag or len(str(item).replace("\n","").strip())>0:
            children.append(item)
        
    return children


In [9]:
page_response = requests.get("https://www.screener.in/company/3MINDIA/", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))

In [10]:
ratios_section = page_content.find("section",attrs={'id':"ratios"})
ratios_table = ratios_section.find("table",attrs={'class':"data-table"})

In [11]:
get_table_simple(ratios_table, is_table_tag=True)

[['Dec 2008',
  'Mar 2010',
  'Mar 2011',
  'Mar 2012',
  'Mar 2013',
  'Mar 2014',
  'Mar 2015',
  'Mar 2016',
  'Mar 2017',
  'Mar 2018',
  'Mar 2019',
  'Mar 2020'],
 ['ROCE %',
  '28%',
  '36%',
  '30%',
  '16%',
  '11%',
  '10%',
  '21%',
  '34%',
  '33%',
  '35%',
  '29%',
  '16%'],
 ['Debtor Days',
  '53',
  '49',
  '60',
  '61',
  '62',
  '56',
  '49',
  '54',
  '63',
  '76',
  '71',
  '67'],
 ['Inventory Turnover',
  '4.82',
  '5.38',
  '4.70',
  '4.43',
  '4.37',
  '4.45',
  '3.90',
  '4.13',
  '4.26',
  '3.97',
  '4.14',
  '4.12']]

In [12]:
targetFile = open('page_content.txt', 'w')
targetFile.write(str(page_content))

#print(page_content)

46949

In [13]:
pAndL_section = page_content.find("section",attrs={'id':"profit-loss"})
pAndL_table = pAndL_section.find("table",attrs={'class':"data-table"})
get_table_simple(pAndL_table, is_table_tag=True)

[['Dec 2008',
  'Mar 2010',
  'Mar 2011',
  'Mar 2012',
  'Mar 2013',
  'Mar 2014',
  'Mar 2015',
  'Mar 2016',
  'Mar 2017',
  'Mar 2018',
  'Mar 2019',
  'Mar 2020',
  'TTM'],
 ['Sales +',
  '742',
  '1087',
  '1191',
  '1405',
  '1574',
  '1742',
  '1840',
  '2140',
  '2363',
  '2556',
  '2809',
  '2765',
  '2342'],
 ['Expenses +',
  '654',
  '948',
  '1035',
  '1283',
  '1463',
  '1630',
  '1636',
  '1803',
  '1994',
  '2076',
  '2301',
  '2425',
  '2188'],
 ['Operating Profit',
  '88',
  '138',
  '156',
  '122',
  '112',
  '112',
  '204',
  '337',
  '369',
  '480',
  '508',
  '340',
  '154'],
 ['OPM %',
  '12%',
  '13%',
  '13%',
  '9%',
  '7%',
  '6%',
  '11%',
  '16%',
  '16%',
  '19%',
  '18%',
  '12%',
  '7%'],
 ['Other Income',
  '11',
  '20',
  '11',
  '5',
  '10',
  '19',
  '14',
  '18',
  '49',
  '44',
  '33',
  '42',
  '42'],
 ['Interest',
  '0',
  '1',
  '2',
  '4',
  '10',
  '15',
  '4',
  '2',
  '2',
  '2',
  '1',
  '4',
  '4'],
 ['Depreciation',
  '7',
  '17',
  '17',